California - doesn't have attribute "Breed" and "Age"

*   California "Age" = "Intake.Date" - "DOB"

*   "Breed" hasn't been check yet, but removed for now.

Age column is in years.


In [22]:
import pandas as pd

austin = pd.read_csv("merged_austin_data.csv")
indiana = pd.read_csv("indiana-animal-data.csv")

#adding "state" attribute - identify which state each pet came from
austin["State"] = "Austin"
indiana["State"] = "Indiana"

print(austin.shape)
print(indiana.shape)

(165271, 27)
(10290, 24)


In [23]:
#rename, drop irrelevant columns
austin = austin.rename(columns={
    'Animal.ID.x': 'Animal.ID',
    'Name.x': 'Name',
    'MonthYear.x': 'MonthYear.Intake',
    'MonthYear.y': 'MonthYear.Outcome',
    'Animal.Type.x': 'Animal.Type',
    'Breed.x': 'Breed',
    'Color.x': 'Color',
    'DateTimeUTC.x': 'Intake.Date',
    'DateTimeUTC.y': 'Outcome.Date',
    'Sex.upon.Outcome': 'Sex',
    'Age.upon.Outcome': 'Age'

})

austin = austin.drop(columns=[col for col in austin.columns if '.y' in col])
austin = austin.drop(columns=['Date.of.Birth', 'Intake.Condition', 'DateTime.x', 'Outcome.Subtype', 'MonthYear.Intake', 'MonthYear.Outcome', 'Sex.upon.Intake', 'Age.upon.Intake', 'Found.Location'])

print(austin.columns)

austin = austin.dropna()

indiana = indiana.rename(columns={
    'id': 'Animal.ID',
    'animalname': 'Name',
    'speciesname': 'Animal.Type',
    'basecolour': 'Color',
    'breedname': 'Breed',
    'intakedate': 'Intake.Date',
    'intakereason': 'Intake.Type',
    'animalage': 'Age',
    'sexname': 'Sex',
    'movementdate': 'Outcome.Date',
    'movementtype': 'Outcome.Type'
})

indiana = indiana.drop(columns=['sheltercode', 'identichipnumber', 'istransfer', 'istrial', 'returndate', 'returnedreason', 'deceaseddate', 'deceasedreason', 'diedoffshelter', 'puttosleep',
       'isdoa','location'], errors='ignore')

indiana = indiana.dropna()

print(indiana.columns)

#dimensions of individual data sets
print(austin.shape)
print(indiana.shape)


Index(['Animal.ID', 'Name', 'Intake.Type', 'Animal.Type', 'Breed', 'Color',
       'Intake.Date', 'Outcome.Type', 'Sex', 'Age', 'Outcome.Date', 'State'],
      dtype='object')
Index(['Animal.ID', 'Intake.Date', 'Intake.Type', 'Name', 'Breed', 'Color',
       'Animal.Type', 'Age', 'Sex', 'Outcome.Date', 'Outcome.Type', 'State'],
      dtype='object')
(117917, 12)
(10243, 12)


In [24]:
#concatenate 3 datasets
common_columns = list(set(austin.columns) & set(indiana.columns))

austin_aligned = austin.reindex(columns=common_columns)
indiana_aligned = indiana.reindex(columns=common_columns)

optipaw = pd.concat([austin_aligned, indiana_aligned], ignore_index=True)

print(optipaw.shape)
print(optipaw.head())

(128160, 12)
  Animal.Type       Age   State     Outcome.Type            Sex Animal.ID  \
0         Dog   7 years  Austin  Return to Owner  Neutered Male   A006100   
1         Dog   6 years  Austin  Return to Owner  Neutered Male   A006100   
2         Dog  10 years  Austin         Transfer  Neutered Male   A047759   
3         Dog  16 years  Austin  Return to Owner  Neutered Male   A134067   
4         Dog  15 years  Austin  Return to Owner  Spayed Female   A141142   

     Name         Color      Intake.Type          Intake.Date  \
0   Scamp  Yellow/White    Public Assist  2014-12-19 10:21:00   
1   Scamp  Yellow/White    Public Assist  2014-03-07 14:26:00   
2    Oreo      Tricolor  Owner Surrender  2014-04-02 15:55:00   
3  Bandit   Brown/White    Public Assist  2013-11-16 09:02:00   
4  Bettie   Black/White            Stray  2013-11-16 14:46:00   

                         Breed         Outcome.Date  
0         Spinone Italiano Mix  2014-12-20 16:35:00  
1         Spinone Italian

In [25]:
#remove duplicate rows
print(optipaw.duplicated().sum())
optipaw = optipaw.drop_duplicates()

#remove rows with missing values
print(optipaw.isnull().sum())
optipaw = optipaw.dropna() #should be 0, NA rows removed when individual data sets were dealt

#check outcome dates are later than intake dates, otherwise remove row
optipaw['Outcome.Date'] = pd.to_datetime(optipaw['Outcome.Date'], errors='coerce')
optipaw['Intake.Date'] = pd.to_datetime(optipaw['Intake.Date'], errors='coerce')

optipaw_valid_rows = optipaw[optipaw['Outcome.Date'] > optipaw['Intake.Date']]
num_valid_rows = optipaw_valid_rows.shape[0]
print(num_valid_rows)

#reorder heading
order = ['Animal.ID', 'Name', 'Animal.Type', 'Breed', 'Sex', 'Color', 'Age', 'Intake.Type', 'Outcome.Type', 'Intake.Date', 'Outcome.Date', 'State']
optipaw_final = optipaw_valid_rows[order]

#change age to unit years
import re

def convert_age_to_years(age_string):
    unit_conversions = {
        'year': 1,
        'month': 1/12,
        'week': 1/52,
        'day': 1/365
    }

    pattern = r'(\d+)\s*(year|month|week|day)s?'
    match = re.match(pattern, age_string.lower())

    if match:
        number = int(match.group(1))
        unit = match.group(2)
        return number * unit_conversions[unit]
    else:
        return None

optipaw_final['Age'] = optipaw_final['Age'].apply(convert_age_to_years)
optipaw_final['Age'] = optipaw_final['Age'].round(2)

print(optipaw_final.head())

optipaw_final.to_csv("/content/optipaw_new.csv", index=False)

72
Animal.Type     0
Age             0
State           0
Outcome.Type    0
Sex             0
Animal.ID       0
Name            0
Color           0
Intake.Type     0
Intake.Date     0
Breed           0
Outcome.Date    0
dtype: int64
124914
  Animal.ID    Name Animal.Type                        Breed            Sex  \
0   A006100   Scamp         Dog         Spinone Italiano Mix  Neutered Male   
1   A006100   Scamp         Dog         Spinone Italiano Mix  Neutered Male   
2   A047759    Oreo         Dog                    Dachshund  Neutered Male   
3   A134067  Bandit         Dog            Shetland Sheepdog  Neutered Male   
4   A141142  Bettie         Dog  Labrador Retriever/Pit Bull  Spayed Female   

          Color   Age      Intake.Type     Outcome.Type         Intake.Date  \
0  Yellow/White   7.0    Public Assist  Return to Owner 2014-12-19 10:21:00   
1  Yellow/White   6.0    Public Assist  Return to Owner 2014-03-07 14:26:00   
2      Tricolor  10.0  Owner Surrender         Tr